<a href="https://colab.research.google.com/github/Larsdegroot/med_Unet/blob/main/run_experiment_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run UNet Training
This notebook is meant to be used in google colab to train a UNet model on the WMH dataset for MRI brain image segmentation.

## Set up environment

In [2]:
# don't restart session it's fine colab is a liar
!pip3 install torch torchvision torchaudio
!pip3 install "lightning[pytorch-extra]"
!pip3 install monai
!pip3 install SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 11.6 MB/s eta 0:00:00


In [3]:
! git clone "https://github.com/Larsdegroot/med_Unet"

Cloning into 'med_Unet'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 185 (delta 6), reused 2 (delta 0), pack-reused 159 (from 1)
Receiving objects: 100% (185/185), 1.24 MiB | 4.83 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [5]:
import zipfile
from pathlib import Path

# Define the target directory
target_dir = Path("med_Unet/data")

# Create the directory if it doesn't exist
target_dir.mkdir(parents=True, exist_ok=True)

# Extract the ZIP files
for dataset in ["WMH", "BraTS"]:
  try:
    with zipfile.ZipFile(f"med_Unet/data/{dataset}.zip", 'r') as zip_ref:
        zip_ref.extractall(target_dir)
    print(f"Extracted the {dataset} dataset.")
  except FileNotFoundError: print(f"No zipfile found for the {dataset} dataset in {target_dir}")
  except zipfile.BadZipFile: print(f"Zipfile for {dataset} not complete, wait for it to upload")


No zipfile found for the WMH dataset in med_Unet/data
No zipfile found for the BraTS dataset in med_Unet/data


## Config rclone

In [ ]:
!wget https://downloads.rclone.org/v1.68.2/rclone-v1.68.2-linux-amd64.deb
!apt install ./rclone-v1.68.2-linux-amd64.deb
!sudo apt-get -y install fuse3

In [ ]:
!rclone config
#!rclone config create onedrive onedrive

In [ ]:
!sudo mkdir /content/onedrive

!rclone mount onedrive:/ /content/onedrive --daemon --vfs-cache-mode full --log-file /tmp/rclone.log --log-level DEBUG

## Run training

In [ ]:
!python3 med_Unet/main.py fit --config med_Unet/configs/default_config_2.yaml --logger tensorboard --log_dir /content/onedrive/logs --name default_config_2

2025-01-08 13:36:51.773672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-08 13:36:51.808785: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-08 13:36:51.819142: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-08 13:36:51.844393: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-08 13:36:53.465682: W tensorflow/comp

## Run Testing

In [ ]:
#!python3 med_Unet/main.py test --config med_Unet/configs/testing_config